In [26]:
# 基本ライブラリ
import os
from pathlib import Path
import re
import sys
import time
import traceback
import warnings
from urllib.request import Request, urlopen

# 外部ライブラリ
from bs4 import BeautifulSoup
import pandas as pd
import pickle
from tqdm import tqdm
from urllib.error import HTTPError, URLError

# Selenium関連のライブラリ
from selenium.webdriver.common.by import By

# # スクリプトの1つ上の階層をsys.pathに追加
# sys.path.append(str(Path(__file__).resolve().parent))
# Jupyter環境でのカレントディレクトリを使う
current_dir = Path.cwd()  # 現在の作業ディレクトリ
sys.path.append(str(current_dir.resolve().parent.parent))



# 自作モジュール
from src.logger_setting import setup_logger
from src.chrome_setting import get_chrome_driver

# pandas worning非表示設定
"""
warning箇所
# 有効な<table>をHTML文字列に変換して、pd.read_htmlに渡す
dfs = pd.read_html(str(valid_tables))
"""
warnings.simplefilter("ignore", FutureWarning)


# ロガーの取得
logger = setup_logger(__name__)

# 定数の定義
HTML_DIR = Path("data", "html")
SAVE_DIR = Path("data", "rawdf")

# URL定義
RACE_DATE_URL_TEMPLATE = (
    "https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
)
RACE_ID_LIST_URL_TEMPLATE = (
    "https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
)
RACE_URL_TEMPLATE = "https://db.netkeiba.com/race/{race_id}"
HORSE_URL_TEMPLATE = "https://db.netkeiba.com/horse/{horse_id}"

# スクレイピング関連定数
HTML_RACE_DIR = HTML_DIR / "race"
HTML_HORSE_DIR = HTML_DIR / "horse"
RAWDF_RACE_FILE_NAME = "race_results.csv"
RAWDF_HORSE_FILE_NAME = "horse_results.csv"
RAWDF_PREPROCESSED_RACE_FILE_NAME = "preprocessed_race_results.csv"
RAWDF_PREPROCESSED_HORSE_FILE_NAME = "preprocessed_horse_results.csv"
LOOP_WAIT_SECONDS = 3  # スクレイピング間の待機時間
FLOM_DATE = "2024-01"  # スクレイピング開始年月
TO_DATE = "2024-11"  # スクレイピング終了年月

# ID抽出関連
COLUMN_RACE_ID = "race_id"
COLUMN_HORSE_ID = "horse_id"
COLUMN_JOCKEY_ID = "jockey_id"
COLUMN_TRAINER_ID = "trainer_id"
COLUMN_OWNER_ID = "owner_id"
COLUMN_RANK = "rank"
COLUMN_WAKUBAN = "wakuban"
COLUMN_UMABAN = "umaban"
COLUMN_SEX = "sex"
COLUMN_AGE = "age"
COLUMN_WEIGHT = "weight"
COLUMN_WEIGHT_DIFF = "weght_diff"
COLUMN_TANSYO = "tansyo"
COLUMN_POPULARITY = "popularity"
COLUMN_IMPOST = "impost"

HORSE_ID_LENGTH = 10
JOCKEY_ID_LENGTH = 5
TRAINER_ID_LENGTH = 5
OWNER_ID_LENGTH = 6

# User-Agent 定義
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

# メッセージ定数
PROCESS_MESSAGE = ":process start"
RACE_ID_PICKLE = "race_id_from202401_to202411.pickle"
RESULT_PICKLE = "result_data_from202401_to202411.pickle"

# エラーメッセージ
ERROR_NO_VALID_TABLE = "HTMLドキュメントに有効な<table>要素が見つかりませんでした。"
ERROR_UNEXPECTED = "予期せぬエラーが発生しました。"
ERROR_INVALID_URL = "無効なURLが指定されました。"

# DataFlame加工処理定数
SEX_MAPPING = {"牡": 0, "牝": 1, "セ": 2}


In [27]:
read_path = Path(sys.path[-1]) / SAVE_DIR / RAWDF_RACE_FILE_NAME
df = pd.read_csv(read_path, sep="\t")

In [28]:
# 列名確認
df.iloc[:,10:20]

,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id
0,1.2,1.0,462(-2),[東] 上原佑紀,2022105244,1197,1192,42033
1,10.2,4.0,452(-2),[東] 武藤善則,2022106999,1169,1064,897009
2,7.9,3.0,416(+6),[西] 牧浦充徳,2022100639,1170,1113,170800
3,5.9,2.0,410(+6),[東] 黒岩陽一,2022105762,1188,1133,180800
4,21.3,5.0,438(-2),[西] 中村直也,2022100660,1199,1186,170800
...,...,...,...,...,...,...,...,...
42307,32.3,9.0,528(-2),[西] 池江泰寿,2020104913,1160,1071,26803
42308,51.0,11.0,464(0),[西] 音無秀孝,2021110123,1200,1002,346803
42309,167.0,15.0,502(-2),[西] 高野友和,2020100160,1216,1117,170800
42310,85.7,14.0,456(0),[西] 坂口智康,2021101090,1204,1170,163002


In [29]:
# df["着順"].astype(int)
df["rank"]=pd.to_numeric(df["着順"],errors="coerce")
df["rank"].value_counts(dropna=False)
df.dropna(subset=["rank"], inplace=True)
df["rank"].value_counts(dropna=False)

rank
1.0     3120
3.0     3116
2.0     3115
4.0     3114
5.0     3110
6.0     3105
7.0     3087
8.0     3001
9.0     2893
10.0    2731
11.0    2519
12.0    2268
13.0    2016
14.0    1784
15.0    1442
16.0    1058
17.0     264
18.0     181
Name: count, dtype: int64

In [30]:
df["馬番"].astype(int)

# 着順	枠番	馬番	馬名	性齢	斤量	騎手	タイム	着差
# 単勝	人気	馬体重	調教師	horse_id	jockey_id	trainer_id	owner_id

0         5
1         2
2         3
3         1
4         4
         ..
42307     7
42308    10
42309     8
42310    15
42311     1
Name: 馬番, Length: 41924, dtype: int32

In [31]:
df["性齢"].str[0].value_counts()

性齢
牡    22530
牝    17447
セ     1947
Name: count, dtype: int64

In [32]:
sex_mapping = {"牡":0,"牝":1,"セ":2}
df["性齢"].str[0].map(sex_mapping).value_counts()

性齢
0    22530
1    17447
2     1947
Name: count, dtype: int64

In [33]:
df["性齢"].str[1:].astype(int)

0        2
1        2
2        2
3        2
4        2
        ..
42307    4
42308    3
42309    4
42310    3
42311    3
Name: 性齢, Length: 41924, dtype: int32

In [34]:
df["単勝"].astype(float)

0          1.2
1         10.2
2          7.9
3          5.9
4         21.3
         ...  
42307     32.3
42308     51.0
42309    167.0
42310     85.7
42311     10.6
Name: 単勝, Length: 41924, dtype: float64

In [35]:
# df["人気"].astype(int)
df["人気"]=pd.to_numeric(df["人気"],errors="coerce")
df["人気"].value_counts(dropna=False)
df.dropna(subset=["人気"], inplace=True)
df["人気"].value_counts(dropna=False)

人気
3.0     3103
1.0     3102
4.0     3100
2.0     3100
5.0     3099
6.0     3089
7.0     3069
8.0     3000
9.0     2886
10.0    2723
11.0    2519
12.0    2286
13.0    2028
14.0    1805
15.0    1461
16.0    1104
17.0     265
18.0     185
Name: count, dtype: int64

In [36]:
df["weight"] = df["馬体重"].str.extract(r"(\d+)").astype(int)

In [37]:
df["weght_diff"] = df["馬体重"].str.extract(r"\((.+)\)").astype(int)

In [40]:
read_path = Path(sys.path[-1]) / SAVE_DIR / RAWDF_PREPROCESSED_RACE_FILE_NAME
df = pd.read_csv(read_path, sep="\t")
df

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,wakuban,umaban,sex,age,weight,weght_diff,tansyo,popularity,impost
0,202401010101,2022105244,1197,1192,42033,1.0,5,5,22530.0,2,462,-2,1.2,1.0,55
1,202401010101,2022106999,1169,1064,897009,2.0,2,2,17447.0,2,452,-2,10.2,4.0,55
2,202401010101,2022100639,1170,1113,170800,3.0,3,3,1947.0,2,416,6,7.9,3.0,55
3,202401010101,2022105762,1188,1133,180800,4.0,1,1,NaN,2,410,6,5.9,2.0,55
4,202401010101,2022100660,1199,1186,170800,5.0,4,4,NaN,2,438,-2,21.3,5.0,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41919,202410030812,2020104913,1160,1071,26803,11.0,4,7,NaN,4,528,-2,32.3,9.0,58
41920,202410030812,2021110123,1200,1002,346803,12.0,6,10,NaN,3,464,0,51.0,11.0,54
41921,202410030812,2020100160,1216,1117,170800,13.0,5,8,NaN,4,502,-2,167.0,15.0,53
41922,202410030812,2021101090,1204,1170,163002,14.0,8,15,NaN,3,456,0,85.7,14.0,50


In [ ]:
read_path = Path(sys.path[-1]) / SAVE_DIR / RAWDF_HORSE_FILE_NAME
df = pd.read_csv(read_path, sep="\t")
df.iloc[:,0:10]

regex_race_class = "|".join(race_class_mapping.keys())
df['レース名'].str.extract(rf'({regex_race_class})')[0].map(race_class_mapping).value_counts(dropna=True)
    # df[COLUMN_RANK] = pd.to_numeric(df["着順"], errors="coerce")
    # df.dropna(subset=[COLUMN_RANK], inplace=True)
    # df[COLUMN_DATE] = pd.to_datetime(df['日付'], format='%Y-%m-%d')
    # df[COLUMN_WEATHER] = df['天気'].map(weather_mapping)
    # df[COLUMN_RACE_TYPE] = df['距離'].str[0].map(race_type_mapping)

    # df[COLUMN_COURSE_LEN] = df['距離'].str.extract(r'(\d+)').astype(int)
    # df[COLUMN_GROUND_STATE] = df['馬場'].map(ground_state_mapping)

    # df[COLUMN_RANK_DIFF] = df['着差'].map(lambda x: 0 if x < 0 else x)
    # df[COLUMN_PRIZE] = df['賞金'].fillna(0)
    # regex_race_class = "|".join(race_class_mapping.keys())
    # df[COLUMN_RACE_CLASS] = df['レース名'].str.extract(rf'({regex_race_class})')[0].map(race_class_mapping)

着差
 0.6     5744
 0.5     5638
 0.4     5575
 0.8     5501
 0.7     5434
         ... 
 14.2       1
 18.7       1
-3.6        1
 24.7       1
-3.3        1
Name: count, Length: 227, dtype: int64